In [42]:
import os
from cryptography.hazmat.primitives import hashes, hmac
import time
from cryptography.hazmat.primitives.twofactor.hotp import HOTP
from cryptography.hazmat.primitives.kdf.scrypt import Scrypt
from cryptography.hazmat.primitives.hashes import SHA1


In [13]:
def hash_password(password: str) -> tuple[bytes, bytes]:

    salt = os.urandom(16)
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1)
    hashed = kdf.derive(password.encode("utf-8"))
    
    return salt, hashed

In [12]:
def verify_password(poizkus:str, salt: int, hashed: bytes) -> bool:
    kdf = Scrypt(salt=salt, length=32, n=2**14, r=8, p=1)
    try:
        kdf.verify(poizkus.encode("utf-8"), hashed)
        return True
    except Exception:
        return False

In [ ]:
## Test

salt, hashed = hash_password("To je moj kuža")
if verify_password("To je moj kuža", salt, hashed):
    print("Geslo je pravo")
else:
    print("Geslo je napačno")

Geslo je pravo


In [31]:
def hotp(secret: bytes, counter: int, digits: int=6):
    passwords = []
    hotp = HOTP(key=secret, length=digits, algorithm=SHA1())
    for i in range(counter):
        passwords.append(hotp.generate(counter=i))
    
    return passwords

In [32]:
## Test
k = os.urandom(64)
print(k)
print(hotp("kuza laja sem pa tja".encode("utf-8"), 10, 8))

b'\xdc\xa2$y\xe5y*\x87<\xa9\xe4\x0f$\xff{\xad\x9a\xe0\x13`\xcd8\xc7\xa5\xcdW\xb28y\xb5\x1e\x0e_\x02zkn~\x9cW\x98\x1b\xe2c\x81\xcf{\xbb\x19\x9a\xe1v\t\xb5\x97\xae\x00|\x08\xe1\x80\x05\xa8f'
[b'28287894', b'27629146', b'16050701', b'09376102', b'76311275', b'35184578', b'90670889', b'67596189', b'25448547', b'27426210']


In [50]:
def totp(secret: bytes, time_step: int = 30, digits: int = 6, t = None):
    passwords = []
    if not t:
        t = (int) (time.time() / time_step)
    hotp = HOTP(key=secret, length=digits, algorithm=SHA1())
    
    passwords.append(hotp.generate(t - 1))
    passwords.append(hotp.generate(t))
    passwords.append(hotp.generate(t + 1))
        
    return passwords

In [51]:
## Test (Napaka)

print(totp("kuza laja sem pa tja".encode("utf-8"), 10, 6))

[b'196846', b'134495', b'809265']


In [ ]:
# Poziv
# Kako naredim 16 mestno št.
def generate_challenge() -> str:
    """Ustvari naključno 6-mestno številko"""
    rnd = os.urandom(4)
    number = int.from_bytes(rnd, byteorder="big")
    number = number & 0x7FFFFFFF  # nastavi MSB na 0 (da bo pozitivno)
    number = number % 1000000  # ostanek pri deljenju z 10^6 da 6-mestno število
    # vrnemo 6 mestni niz: če je število manj-mestno, ga podložimo z ničlami
    return "{0:06d}".format(number)

# ali

def generate_challenge_test() -> str:
    num = os.urandom(16)
    return num.hex()

# Odziv
def generate_response_hmac_256(key: bytes, chalange: str) -> str:
    h = hmac.HMAC(key, hashes.SHA256())
    h.update(chalange.encode("utf-8"))
    return h.finalize().hex()

In [49]:
## Test
# Kaj dejansko dela ta poziv odziv?

s = generate_challenge()
print(s)
print(generate_response_hmac_256("kuza laja sem pa tja".encode("utf-8"), s))

s = generate_challenge_test()
print(s)
print(generate_response_hmac_256("kuza laja sem pa tja".encode("utf-8"), s))

724831
9db0ba22053c06dfb1b9ef60ac52e6cc59af4685f63e767e56754e31305f53c7
ada9b13944429f6197c72990854cd0f6
315c934b4d8dbbfe569f27dea52abb17ffa72fb223109e444eb9c7d9c8dcb607
